<a href="https://colab.research.google.com/github/vin-thomas/transcript/blob/main/Q%26A_BM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORTS**

In [1]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('openai')

In [ ]:
!pip -q install openai tiktoken cohere

In [3]:
import openai

In [64]:
!sudo apt-get update && sudo apt install build-essential libpoppler-cpp-dev pkg-config python3-dev

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [47.6 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [632 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,027 kB]
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,535 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:13 https://pp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
# models
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

In [6]:
from openai import OpenAI
client = OpenAI()

In [65]:
!pip install pdftotext

In [67]:
import pdftotext

In [ ]:
!pip install rank_bm25

In [10]:
from rank_bm25 import BM25Okapi

In [11]:
# imports
import ast  # for converting embeddings saved as strings back to arrays
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
from scipy import spatial  # for calculating vector similarities for search

In [12]:
import string

**HELPER FUNCTIONS**

In [126]:
def text_extraction(path):
  with open (path, 'rb') as f:
    pdf = pdftotext.PDF(f)
  text = []
  for txt in pdf:
    text.append(txt)
  return text

In [127]:
def text_extractionTXT(path):
  with open (path, 'r') as f:
    pdf = f.read()
  text = []
  # for txt in pdf:
  text.append(pdf)
  return text

In [99]:
import nltk

# Download the punkt tokenizer if not already installed
nltk.download('punkt')

def split_into_sentences(text):
    sentences = nltk.sent_tokenize(text)
    return sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

In [16]:
def remove_punct(input_string, string):
  return input_string.translate(str.maketrans('', '', string.punctuation)).lower()

**CREATE THE CORPUS**

In [128]:
import os
import re
texts = []
fname = []
def find_pdfs(folder_path):
    pdf_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                pdf_files.append(os.path.join(root, file))
    return pdf_files

# Provide the path of the folder you want to search for PDF files
folder_path = '/content/drive/MyDrive/fin'

pdf_paths = find_pdfs(folder_path)
for pdf_path in pdf_paths:
  fn = re.search(r'/content/(?P<extracted_part>.*?)\.pdf$', pdf_path)
  if fn:
    fname.append(fn.group('extracted_part'))
  texts.append(text_extraction(pdf_path))

In [131]:
import os
# Replace these paths with your desired values
root = '/content/drive/MyDrive/fin/'
fnames = []
kttexts = []
comp_ls = os.listdir(root)
for name in comp_ls:
  years = os.listdir(root+name+'/')
  for year in years:
    qrtrs = ['1','2','3','4']
    for qrtr in qrtrs:
      if os.path.exists(root+name+'/'+year+'/'+qrtr+'/'+'keytakeaways'):
        fnames.append(root+name+'/'+year+'/'+qrtr+'/'+'keytakeaways'+'/'+name+'.txt')
for path in fnames:
  # print(path)
  kttexts+=text_extractionTXT(path)

In [ ]:
# @title Glob
import glob

directory = '/content'
pattern = directory + '/*.pdf'

texts = []

file_list = glob.glob(pattern)

for file in file_list:
  print(file)
  texts += text_extraction(file)

In [133]:
ktaways = []
for item in kttexts:
  senti = []
  sent = item.split('\n')
  for i in sent:
    if i != '':
      senti.append(i)
  ktaways.append(senti)

In [137]:
texts

[['CORPORATE PARTICIPANTS\nT V Narendran, CEO & MD – Tata Steel Limited\nKoushik Chatterjee, ED & CFO – Tata Steel Limited\nSamita Shah, VP Corporate Finance, Treasury, & Risk Management – Tata Steel Limited\n\nCONFERENCE CALL PARTICIPANTS\nAlok Deora, Motilal Oswal\nAmit Murarka, Axis Capital\nAshish Jain, Macquarie\nKirtan Mehta, BoB Capital\nMudit Bhandari, IIFL\nPinakin Parekh, JP Morgan\nPrashanth Kota, Emkay Global\nRitesh Shah, Investec\nSatyadeep Jain, Ambit Capital\nSumangal Nevatia, Kotak Securities\nVishnu Kumar, Avendus Spark Capital\n\nPRESENTATION\nOperator:\nLadies and gentlemen, good day, and welcome to the Tata Steel analyst call. Please note that this meeting is being\nrecorded. All the attendees, audio and video has been disabled from the backend and will be enabled subsequently. I\nwould now like to hand the conference over to Ms. Samita Shah. Thank you and over to you ma’am.\nSamita Shah: VP CFTRM – Tata Steel Limited\nThank you, Kinshuk. Good afternoon, good morni

In [136]:
sentences = []
for item in texts:
  sent += split_into_sentences(item)
  sentences.append(sent)

TypeError: ignored

In [ ]:
sentences

In [25]:
corpus = []
# Iterate over the original list, considering only elements up to the 5th from the end
for i in range(len(sentences) - 4):
    # Concatenate the current element with the next four elements and append to the new list
    concatenated_string = ' '.join(sentences[i:i+5])
    corpus.append(concatenated_string)

**CONTEXT USING BM25**

In [26]:
cleaned_corpus = []
for doc in corpus:
  cleaned_doc = remove_punct(doc, string)
  cleaned_corpus.append(cleaned_doc)

In [27]:
tokenized_clean_corpus = []
for doc in cleaned_corpus:
  doc = doc.split ()
  tokenized_clean_corpus.append(doc)

In [28]:
bm25 = BM25Okapi(tokenized_clean_corpus)

In [29]:
def gen_input_1(query):
  tokenized_query = remove_punct(query, string).split()
  top_5_results = bm25.get_top_n(tokenized_query, corpus, n=5)
  input_1 = ' '
  for item in top_5_results:
    if num_tokens(input_1)< 1024:
      input_1 += item
    else:
      break
  print (num_tokens(input_1))
  return input_1


**CONTEXT USINGS SEMANTIC SEARCH**

In [30]:
df = pd.DataFrame (corpus, columns =['sentences'])

In [31]:
df['embeddings'] = df.sentences.apply(lambda x: openai.embeddings.create(input=x, model= EMBEDDING_MODEL).data[0].embedding)
df.to_csv('/content/embeddings.csv')

In [32]:
# search function
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100
) -> tuple[list[str], list[float]]:
    """Returns a list of strings and relatednesses, sorted from most related to least."""
    query_embedding_response = openai.embeddings.create(
        model=EMBEDDING_MODEL,
        input=query,
    )
    query_embedding = query_embedding_response.data[0].embedding
    strings_and_relatednesses = [
        (row["sentences"], relatedness_fn(query_embedding, row["embeddings"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings [:top_n], relatednesses[:top_n]


In [33]:
def gen_input_2(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    strings, relatednesses = strings_ranked_by_relatedness(query, df)
    message = " "
    for string in strings:
      if num_tokens (message) < token_budget:
        message += string
      else:
        break
    return message

**GENERATE RESPONSE USING LLM**

In [34]:
def llm_response(query):

  input_1 = gen_input_1(query)
  input_2 = gen_input_2(query, df, GPT_MODEL, 1024)

  message = f'''Please answer the query in short and simple setences - {query}, based on the following extracts: '''

  message += input_1 + '\n\n' + input_2
  messages = [
      {"role": "system", "content": "You answer questions based on the context provided."},
      {"role": "user", "content": message},
    ]

  response = client.chat.completions.create(
      model= GPT_MODEL,
      messages=messages,
      temperature=0
      )
  response = response.choices[0].message.content
  return response

In [ ]:
answer = llm_response('What did Bala Murali Krishna say?')

In [38]:
print (answer)

Bala Murali Krishna asked about the guidance for 1200 buses for FY2024 and mentioned that only around 75 buses were delivered in the current quarter. He also asked for a breakup of the 5000 balance tenders from which state and how much it is. Sharat Chandra responded that there are about 5150 orders from MSRC and 2080 orders from BEST in Mumbai.
